# **VİDEOLARI İNDİRMEK**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install yt-dlp
!pip install unidecode
!pip install ultralytics==8.0.196

In [ ]:
import yt_dlp
import threading
import unidecode

def sanitize_filename(title):
    # Türkçe karakterleri İngilizce karşılıklarıyla değiştir
    return unidecode.unidecode(title)

def download_video(video_url):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]',
        'outtmpl': '%(title)s.%(ext)s',
        'restrictfilenames': True,  # Geçersiz karakterleri kaldır
        'noplaylist': True,
        'external_downloader': 'aria2c',
        'external_downloader_args': ['-x', '16', '-k', '1M']
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)  # Önce video başlığını al
        title = info_dict.get('title', None)
        if title:
            sanitized_title = sanitize_filename(title)
            ydl_opts['outtmpl'] = f'{sanitized_title}.%(ext)s'  # Başlıkta Türkçe karakterler yerine İngilizce karşılıklarını kullan
            with yt_dlp.YoutubeDL(ydl_opts) as ydl_updated:
                ydl_updated.download([video_url])

def download_videos_parallel(video_urls):
    threads = []
    for url in video_urls:
        thread = threading.Thread(target=download_video, args=(url,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

video_urls = [
    'https://www.youtube.com/watch?v=oh9osWZbbyw',
    'https://www.youtube.com/watch?v=hvZ2zg8h0Fc',
    'https://www.youtube.com/watch?v=QbcQO4QhGfI',
    'https://www.youtube.com/watch?v=-ychJNGqXjE&t=2s'
]

download_videos_parallel(video_urls)


# **ISI HARİTASINI OLUŞTURMAK**

*Creating the heat map*

---



In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import os
import glob
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.spatial import ConvexHull
from scipy.spatial.distance import pdist
import re
from sklearn.mixture import GaussianMixture

# Kullanıcıdan maçın başlama saniyesini isteyen fonksiyon
def get_user_input(name):
    while True:
        try:
            user_input = int(input("Lütfen " + name + " maçı için başlangıç saniyesi girin: "))
            return user_input
        except ValueError:
            print("Geçersiz giriş. Lütfen bir sayı girin.")

def field_lines(imgLogo,x1,y1,x2,y2,colored_heatmap):
  # Hedef bölgenin boyutlarını hesaplayın
  target_width = x2 - x1
  target_height = y2 - y1

  # Yerleştirilecek resmi hedef bölgeye sığacak şekilde yeniden boyutlandırın
  resized_overlay = cv2.resize(imgLogo, (target_width, target_height))
  colored_heatmap[y1:y2, x1:x2] = resized_overlay

  # çizgi çizme
  cv2.rectangle(colored_heatmap, (7,79), (389,298), (255, 255, 255),2 )
  cv2.line(colored_heatmap, (198,79), (198,298), (255, 255, 255),2 )
  cv2.circle(colored_heatmap, (198,189), 30, (255, 255, 255),2 )
  cv2.circle(colored_heatmap, (198,189), 4, (255, 255, 255),-1 )
  # kaleler
  cv2.rectangle(colored_heatmap, (7,158), (21,219), (255, 255, 255),2 )
  cv2.rectangle(colored_heatmap, (389,158), (375,219), (255, 255, 255),2 )
  cv2.circle(colored_heatmap, (39,189), 2, (255, 255, 255),-1 )
  cv2.circle(colored_heatmap, (358,189), 2, (255, 255, 255),-1 )
  # ceza sahasi
  cv2.rectangle(colored_heatmap, (7,128), (56,249), (255, 255, 255),2 )
  cv2.rectangle(colored_heatmap, (389,128), (340,249), (255, 255, 255),2 )

# Homography estimation için optimizasyon yapan fonksiyon
def homography_residuals(params, src_pts, dst_pts):
    H = params.reshape((3, 3))

    src_pts_h = np.hstack([src_pts, np.ones((src_pts.shape[0], 1))])
    dst_pts_proj = (H @ src_pts_h.T).T

    # Z bileşeni sıfır olan durumlar için kontrol ekleyin
    with np.errstate(divide='ignore', invalid='ignore'):
        dst_pts_proj /= dst_pts_proj[:, 2].reshape(-1, 1)

    # Nan veya inf değerleri sıfır olarak değiştirin
    dst_pts_proj = np.nan_to_num(dst_pts_proj, nan=0.0, posinf=0.0, neginf=0.0)

    residuals = dst_pts - dst_pts_proj[:, :2]
    return residuals.flatten()

# Oyuncu koordinatlarını oluşturan fonksiyon
def transform_player_positions(player_positions, H):
    player_positions = np.array(player_positions, dtype=np.float32)
    player_positions = player_positions.reshape(-1, 1, 2)
    transformed_positions = cv2.perspectiveTransform(player_positions, H)
    return transformed_positions.reshape(-1, 2)

# Kümelenme kontrolü için oyuncu koordinatlarını hazırlayan fonksiyon
def flatten_and_validate_points(points):
    cleaned_points = []
    for point in points:
        if isinstance(point, np.ndarray) and point.ndim == 2:
            for sub_point in point:
                if len(sub_point) == 2 and np.all(np.isfinite(sub_point)):
                    cleaned_points.append(sub_point)
    return np.array(cleaned_points, dtype=float)

# Küme kontrolü yapan fonksiyon
def check_point_distribution(points, threshold=0.1, ratio_threshold=0.5):
    cleaned_points = flatten_and_validate_points(points)

    if len(cleaned_points) < 3:
        return False

    try:
        hull = ConvexHull(cleaned_points)
    except Exception as e:
        return False

    hull_ratio = len(hull.vertices) / len(cleaned_points)
    if hull_ratio < ratio_threshold:
        return False

    distances = pdist(cleaned_points)
    dist_variance = np.var(distances)

    if dist_variance < threshold:
        return False
    return True

# Video isminden takım isimlerini çıkartan fonksiyon
def extract_team_names(video_name):
    name, _ = os.path.splitext(video_name)

    match = re.search(r'\b([\w\s]+?)\s+VS\s+(.+)\b', name, re.IGNORECASE)

    if match:
        team1 = match.group(1).strip()
        team2 = match.group(2).strip()

        team1 = re.sub(r'\bSEZON\b', '', team1, flags=re.IGNORECASE).strip()
        team2 = re.sub(r'\bSEZON\b', '', team2, flags=re.IGNORECASE).strip()

        return team1, team2
    else:
        return None, None

# OYUN YÜZDELERİ İÇİN FONKSİYONLAR
# Belli bir bölgenin dominant rengini döndüren fonksiyon
def get_dominant_color(roi):
    return np.array(cv2.mean(roi)[:3])

# Oyuncuların üst(forma) bölgelerinin koordinatlarını döndüren fonksiyon
def get_roi(frame, box):
    x1, y1, x2, y2 = map(int, box[:4])
    x1_ = x1 + ((x2 - x1) / 4)
    x2_ = x2 - ((x2 - x1) / 4)
    y1_ = y1 + ((y2 - y1) / 4)
    y2_ = y2 - ((y2 - y1) / 2)
    return frame[int(y1_):int(y2_), int(x1_):int(x2_)]

# Verilen bir futbolcunun takımını döndüren fonksiyon
def determine_team(dominant_color, team_colors):
    distances = [np.linalg.norm(dominant_color - color) for color in team_colors]
    return np.argmin(distances) +1

# Oyuncuların koordinatlarını liste halinde döndüren fonksiyon
def process_frame(results):
    bboxes = []
    player_positions = []
    for result in results:
        for box in result.boxes:
            if(box.cls!=1):
                bboxes.append(box.xyxy.cpu().numpy().flatten())
                player_positions.append([int(box.xyxy[0][0])+ (int((box.xyxy[0][2] - box.xyxy[0][0]) / 2)), int(box.xyxy[0][3])])

    return bboxes, player_positions

# Oyuncuları takım renklerine ayıran fonksiyon
def cluster_colors(color_samples, n_clusters=2):
    gmm = GaussianMixture(n_components=n_clusters, random_state=0)
    gmm.fit(color_samples)
    return gmm.means_

# Top ve oyuncular arasındaki mesafeyi hesaplayan fonksiyon
def calculate_distance(box1, box2):
    x1_1, y1_1, x2_1, y2_1 = map(int, box1)
    center_x1 = (x1_1 + x2_1) / 2
    center_y1 = (y1_1 + y2_1) / 2

    x1_2, y1_2, x2_2, y2_2 = map(int, box2)
    bottom_center_x2 = (x1_2 + x2_2) / 2
    bottom_center_y2 = y2_2

    distance = np.linalg.norm(np.array([center_x1, center_y1]) - np.array([bottom_center_x2, bottom_center_y2]))

    return distance, (int(center_x1), int(center_y1)), (int(bottom_center_x2), int(bottom_center_y2))

# Topun hangi takıma en yakın olduğunu döndüren fonksiyon
def annotate_frame(frame, bboxes, team_colors, results):
    a = False
    top_box = None
    for result in results:
        for box in result.boxes:
            if int(box.cls) == 0:
                top_box = box.xyxy.cpu().numpy().flatten()
                a = True
                break  # ilk topu bulduktan sonra umursamıyoruz

    player_index = 0
    min_distance = 10000
    teamf = -1
    dominant_team = -1
    team_counts = {1: 0, 2: 0}
    for box in bboxes:
        roi = get_roi(frame, box)
        dominant_color = get_dominant_color(roi)
        team = determine_team(dominant_color, team_colors)
        team_counts[team] += 1
        if a and top_box is not None:
            distance, center1, center2 = calculate_distance(top_box, box)
            if(distance < min_distance):
                min_distance = distance
                teamf = team
        player_index += 1
    dominant_team = max(team_counts, key=team_counts.get)
    return teamf, dominant_team

# Ana fonksiyon
def heatmap(video_name, paths, start_time_sec, half, side1, side2):
  # Bu kısımda modelleri yüklüyoruz
  modelPlayer = YOLO(paths[1]) # oyuncuları buluyor
  modelBall = YOLO(paths[5]) # yalnızca topu buluyor
  modelField1 = YOLO(paths[2]) # kaleleri ve orta sahayı buluyor
  modelField2 = YOLO(paths[3]) # ceza sahasını buluyor

  cap = cv2.VideoCapture(video_name)
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

  # Başlangıç ve bitiş frame'lerini hesapla
  start_frame = int(start_time_sec * fps)
  end_frame = (total_frames - start_frame) // 2 if half else start_frame
  start_frame = (total_frames - start_frame) // 2 if not half else start_frame

  # Başlangıç frame'ine git
  cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

  # Takım renklerini birbirinden ayırmak için maç başlangıcından itibaren 20 kadar frame örnek olarak alınıyor
  frame_interval = fps * 20
  example_frames = []
  for i in range(int(frame_interval)):
      ret, frame = cap.read()
      if not ret:
          break
      if i % (fps * 2) == 0:  # Her 2 saniyede bir frame al
          example_frames.append(frame)

  color_samples = []
  for frame in example_frames:
      results = modelPlayer.predict(source=frame,verbose=False)
      bboxes, w = process_frame(results)
      for box in bboxes:
          roi = get_roi(frame, box)
          dominant_color = get_dominant_color(roi)
          color_samples.append(dominant_color)
  team_colors = cluster_colors(color_samples)
  cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

  # Isı haritasının ilk hali burada oluşturuluyor
  base_image_path = paths[0]
  base_image = cv2.imread(base_image_path)
  heatmap_size = (base_image.shape[0], base_image.shape[1])
  heatmap = np.zeros(heatmap_size, dtype=np.float32)

  # Hedef saha resmindeki bütün referans noktalarının koordinatları
  realField = {
    'Kale_L alt': (11, 230), 'Kale_L üst': (11, 133), 'Kale_R alt': (607, 230), 'Kale_R üst': (607, 133),
    'mid_ust': (308, 13), '1_L': (11, 287), '2_L': (11, 75), '3_L': (96, 75), '4_L': (96, 287),
    '1_R': (607, 75), '2_R': (607, 287), '3_R': (520, 287), '4_R': (520, 75), 'Kose_L': (11, 347),
    'Kose_R': (607, 347), 'Kose_R_ust': (607, 13), 'Kose_L_ust': (11, 13), 'mid sağ': (359, 179),
    'mid sol': (258, 179), 'mid üst': (308, 130), 'mid alt': (308, 230)
  }
  model_dict = {
    'modelField1': modelField1,
    'modelField2': modelField2
  }

  fieldPoints = ["1_L", "2_L", "3_L", "4_L", "1_R", "2_R", "3_R", "4_R", "Kose_L", "Kose_R", "Kose_R_ust", "Kose_L_ust", "mid_ust"] # bunlar koordinatları direkt alınacak noktalar
  containsKale, containsMidUst, determine = False, False, True # framede kale, mid_ust olup olmadığını gösteriyorlar
  classKale, team1, team2 = -1, 0, 0
  team1_name, team2_name = "", ""

  frame_number = start_frame
  frame_skip = int(fps)

  while cap.isOpened():
      fieldDict = {}
      ret, frame = cap.read()

      if not ret or (half and frame_number > end_frame): # ret düzgün değilse ya da fonksiyon ilk yarı için dönüyorsa ve ilk yarı bitmişse looptan çıkıyor
          break

      if frame_number % frame_skip != 0: # saniye başına bir tane frame üzerinde işlem yapılması için bu kısım
        frame_number+=1
        continue

      frame_number += 1
      try:
         frame = cv2.GaussianBlur(frame, (5, 5), 0) # görüntü üzerindeki kalabalığı azaltmak ve daha pürüzsüz bir hale getirmek için gaussian blur uyguluyoruz
      except Exception as e:
        continue
      # modeller kullanılarak frame üzerinde tahminler yapılıyor
      resultsPlayer = modelPlayer.predict(source=frame, verbose=False)
      resultsBall = modelBall.predict(source=frame, verbose=False)
      results2 = modelField1.predict(source=frame, verbose=False)
      results3 = modelField2.predict(source=frame, verbose=False)

      # topun framede hangi takımın kontrolünde olduğuna karar verip o takıma bir puan ekliyor
      boxes, w = process_frame(resultsPlayer)
      team, dominant_team = annotate_frame(frame, boxes, team_colors, resultsBall)
      if team != -1:
        team1 += (team == 1)
        team2 += (team == 2)

      # takım isimlerini doğru şekilde optime ediyor
      if(determine and ('Kale_L alt' in fieldDict or 'Kale_R alt' in fieldDict)):
        team, dominant_team = annotate_frame(frame, boxes, team_colors, resultsPlayer)
        if 'Kale_L alt' in fieldDict:
            team1_name, team2_name = (side1, side2) if dominant_team == 1 else (side2, side1)
            determine = False
        elif 'Kale_R alt' in fieldDict:
            team1_name, team2_name = (side2, side1) if dominant_team == 1 else (side1, side2)
            determine = False

      # bu kısımda saha kısımlarını bulan modellerin sonuçları birleştiriliyor
      combined_results = []
      combined_scores = []
      combined_classes = []
      combined_models = []
      for param2 in results2:
          for box in param2.boxes:
              combined_results.append(box.xyxy.cpu().numpy().flatten())
              combined_scores.append(box.conf.cpu().numpy())
              combined_classes.append(box.cls.cpu().numpy())
              combined_models.append('modelField1')
              if int(box.cls) == 0 or int(box.cls) == 1:
                  containsKale = True
                  classKale = int(box.cls)

      for param3 in results3:
          for box in param3.boxes:
              combined_results.append(box.xyxy.cpu().numpy().flatten())
              combined_scores.append(box.conf.cpu().numpy())
              combined_classes.append(box.cls.cpu().numpy())
              combined_models.append('modelField2')
              if(int(box.cls)==10):
                  containsMidUst=True

      # referans noktalarını fieldDict içine yerleştirmek için
      for box, score, cls, model_name in zip(combined_results, combined_scores, combined_classes, combined_models):
          x1, y1, x2, y2 = map(int, box)

          # bu kısımda bulunan objenin ne olduğunu buluyoruz
          label = model_dict[model_name].names[int(cls.item())]

          # o frame için bulunan bütün referans noktaları fieldDictin içine kaydediliyor
          if label in fieldPoints:
              fieldDict[label] = x1 +((x2 - x1) / 2), y1 + ((y2 - y1) / 2)
          elif label == "Kale_L" or label == "Kale_R":
              if label == "Kale_L":
                  fieldDict[label + " alt"] = (x1, y2)
                  fieldDict[label + " üst"] = (x2, y2)
              else:
                  fieldDict[label + " üst"] = (x1, y2)
                  fieldDict[label + " alt"] = (x2, y2)
          else:
              if containsKale and not containsMidUst:
                  if classKale == 0:
                      fieldDict[label + " sol"] = (x1, y1+ ((y2 - y1) / 2))
                  else:
                      fieldDict[label + " sağ"] = (x2, y1+ ((y2 - y1) / 2))
              else:
                  fieldDict[label + " sağ"] = (x2, y1 + ((y2 - y1) / 2))
                  fieldDict[label + " sol"] = (x1, y1 + ((y2 - y1) / 2))
                  fieldDict[label + " üst"] = (x1 + ((x2 - x1) / 2), y1)
                  fieldDict[label + " alt"] = (x1 + ((x2 - x1) / 2), y2)

      # bu kısım homography estimationı daha garanti hale getirmek için genel değişmeyen noktaları framein konumuna göre fieldDicte eklemeye yarıyor
      if('mid sağ' not in fieldDict and 'mid sol' not in fieldDict and 'mid_ust' not in fieldDict):
              if('Kale_R alt' in fieldDict):
                  if('Kose_R' in fieldDict):
                      fieldDict['mid sağ'] = (1,800)
                  else: fieldDict['mid sağ'] = (1,600)
              elif('Kale_L alt' in fieldDict):
                  if('Kose_L' in fieldDict):
                      fieldDict['mid sol'] = (1913,586)
                  else: fieldDict['mid sol'] = (1914,715)

      # homography estimation yaptığımız kısım
      common_keys = set(realField.keys()).intersection(fieldDict.keys())
      if(len(common_keys)>=4): # dörtten az nokta varsa estimation mümkün olmadığı için frame geçiliyor

          source = np.array([fieldDict[key] for key in common_keys],dtype= np.float32)
          destination = np.array([realField[key] for key in common_keys],dtype= np.float32)
          homography_matrix, mask = cv2.findHomography(source, destination, method=cv2.RANSAC, ransacReprojThreshold=5.0)

          if(homography_matrix is not None): # dört nokta olmasına rağmen matrix none olarak dönerse de frame geçiliyor
              # homography matriksini optimize ettiğimiz kısım burası
              try:
                  initial_params = homography_matrix.flatten()
                  result = least_squares(
                          homography_residuals,
                          initial_params,
                          args=(source, destination),
                          method='lm'
                          )
                  homography_matrix = result.x.reshape((3, 3))
              except (ValueError, cv2.error) as e:
                  if('mid alt' not in fieldDict): # mid_altın olduğu durumlarda optimize edilemese bile asıl matriksi kullanarak devam ediyoruz
                    continue

              # transforme edilmiş oyuncu koordinatlarını bulduğumuz ve ısı haritasına eklediğimiz kısım
              try:
                  boxes_, player_positions = process_frame(resultsPlayer)
                  player_positions = np.array(player_positions,np.float32)
                  transformed_points = cv2.perspectiveTransform(np.float32([player_positions]), homography_matrix)
                  # bulunan noktalar arasında yığılma olmuş mu kontrol ettiğimiz kısım
                  if(not check_point_distribution(transformed_points)):
                    continue

                  # ısı haritasına ekleme yaptığımız kısım
                  for pos in transformed_points[0]:
                    if 0 <= int(pos[0]) < heatmap_size[1] and 0 <= int(pos[1]) < heatmap_size[0]:
                      heatmap[int(pos[1]), int(pos[0])] += 1
              except cv2.error as e:
                  continue
      containsKale, containsMidUst = False, False
      classKale = -1

  ########## HEATMAP ##########
  # Normalize the heatmap
  heatmap /= np.max(heatmap)
  # Apply Gaussian blur to enlarge the hot spots
  heatmap_blurred = cv2.GaussianBlur(heatmap, (0, 0), sigmaX=20, sigmaY=20, borderType=cv2.BORDER_REPLICATE)
  # Normalize the blurred heatmap again
  heatmap_blurred /= np.max(heatmap_blurred)
  # Generate the heatmap image
  # Custom colormap tanımlamaq
  colors = [
      (0.0, (0/255, 102/255, 0/255)),    # 0.0-0.2 arası (52,235,58)
      (0.2, (0/255, 255/255, 0/255)),   # 0.2-0.4 arası (143,235,52)
      (0.4, (255/255, 255/255, 0/255)),   # 0.4-0.6 arası (207,242,80)
      (0.6, (255/255, 128/255, 0/255)),   # 0.6-0.8 arası (242,148,80)
      (0.8, (255/255, 0/255, 0/255)),    # 0.8-1.0 arası (212,40,40)
      (1.0, (153/255, 0/255, 0/255))     # 1.0 (aynı renkte bitirme)
  ]
  heat_map = ""
  n_bins = 100  # Colormap'in keskinliğini artırır
  custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=n_bins)
  if(half):
    plt.imshow(heatmap_blurred, cmap=custom_cmap, interpolation='nearest')
    plt.colorbar()
    plt.title('ISI HARITASI')
    plt.axis('off')  # Eksenleri gizle
    plt.savefig('heatmap1.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    colored_heatmap = cv2.imread('heatmap1.png')
    heat_map = "heatmap1.png"
  else:
    plt.imshow(heatmap_blurred, cmap=custom_cmap, interpolation='nearest')
    plt.colorbar()
    plt.title('ISI HARITASI')
    plt.axis('off')  # Eksenleri gizle
    plt.savefig('heatmap2.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    colored_heatmap = cv2.imread('heatmap2.png')
    heat_map = "heatmap2.png"


  #LOGO
  imgLogo= cv2.imread(paths[4])
  x1, y1 = 10, 10
  x2, y2 = 60, 60

  field_lines(imgLogo, x1,y1.x2,y2,colored_heatmap)

  #TAKIM ISIMLERI VE YUZDELER
  side1_percentage ="" , side2_percentage =""
  if(team1_name== side1):
      side1_percentage = (team1 / (team1 + team2))*100
      side2_percentage = (team2 / (team1 + team2))*100
  else:
      side2_percentage = (team1 / (team1 + team2))*100
      side1_percentage = (team2 / (team1 + team2))*100

  # left side
  cv2.putText(colored_heatmap, side1, (7,320),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1, cv2.LINE_AA)
  cv2.putText(colored_heatmap, "% " +f"{side1_percentage:.2f}" , (7,335),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1, cv2.LINE_AA)
  # right side
  text = side2
  x, y = 389, 320
  (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
  start_x = x - text_width
  cv2.putText(colored_heatmap, text, (start_x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1, cv2.LINE_AA)
  cv2.putText(colored_heatmap,  "% " + f"{side2_percentage:.2f}", (start_x, 335), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1, cv2.LINE_AA)


  #MACIN ZAMANI
  text2 = 'ILK YARI' if half else 'IKINCI YARI'

  x2, y2 = 198, 350  # Metnin ortasının denk geleceği koordinatlar
  (text_width, _), _ = cv2.getTextSize(text2, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
  start_x = x2 - (text_width // 2)
  cv2.putText(colored_heatmap, text2, (start_x, y2), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1, cv2.LINE_AA)

  # SONUCU KAYDETME
  filename = side1 + ' VS ' + side2 + (' ilk_yari.png' if half else ' ikinci_yari.png')
  full_path = os.path.join(paths[6], filename)
  cv2.imwrite(full_path, colored_heatmap)
  os.remove(heat_map)




In [ ]:
import os
import glob

paths = []
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/model/player_detect/bestPlayer.pt") # oyuncu ve top tespiti yapan .pt dosyası için path
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/model/player_detect/bestBall.pt") # top tespiti yapan .pt dosyası için path
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/model/field_detect/bestKale.pt") # kale ve orta sahayı tespit eden .pt dosyası için path
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/model/field_detect/bestCezaSahası.pt") # ceza sahasını tespit eden .pt dosyası için path
paths.append('/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/saha.jpg') # base saha.jpg için path
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/kırmızıLogo.jpg") # logo için path
paths.append("/content/drive/MyDrive/Staj Futbol Projesi/ısı haritası/sonuçlar") # sonuçların kaydedileceği path

mp4_files = glob.glob(os.path.join("/content", '*.mp4'))
for video in mp4_files:
  team1, team2 = extract_team_names(os.path.basename(video))
  start_time = get_user_input(os.path.basename(video))
  heatmap(video, paths, start_time, True, team1, team2)
  print(team1 + " ve " + team2 +" maçının ilk yarı analizi sona erdi")
  heatmap(video, paths, start_time, False, team2, team1)
  print(team1 + " ve " + team2 +" maçının ikinci yarı analizi sona erdi")


Lütfen ELITLIG 11.SEZON AKROM CITY VS BEYLERBEYI.mp4 maçı için başlangıç saniyesi girin: 10
AKROM CITY ve BEYLERBEYI maçının ilk yarı analizi sona erdi
AKROM CITY ve BEYLERBEYI maçının ikinci yarı analizi sona erdi
Lütfen ELITLIG 11.SEZON BEYLERBEYI VS TOKI TEXAS.mp4 maçı için başlangıç saniyesi girin: 14
BEYLERBEYI ve TOKI TEXAS maçının ilk yarı analizi sona erdi
BEYLERBEYI ve TOKI TEXAS maçının ikinci yarı analizi sona erdi
Lütfen ELITLIG 11.SEZON ANADOLU TAT 1071 VS ESERTEPE.mp4 maçı için başlangıç saniyesi girin: 16
ANADOLU TAT 1071 ve ESERTEPE maçının ilk yarı analizi sona erdi
ANADOLU TAT 1071 ve ESERTEPE maçının ikinci yarı analizi sona erdi
Lütfen ELITLIG 11.SEZON NORTH ANKARA VS TAXI.mp4 maçı için başlangıç saniyesi girin: 20
NORTH ANKARA ve TAXI maçının ilk yarı analizi sona erdi
NORTH ANKARA ve TAXI maçının ikinci yarı analizi sona erdi
